# EDA 시각화 코드
결측치 및 이상치 탐색

In [3]:
import matplotlib

matplotlib.use("Agg")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
import regex
import warnings

warnings.filterwarnings("ignore")

## 결측치 및 이상치 시각화 클래스

In [10]:
class EDAVisualizer:

    def __init__(self, data_path):
        self.data_path = data_path
        self.users = None
        self.books = None
        self.train = None
        self.test = None

    def load_data(self):
        print("> 데이터 로딩 중")
        self.users = pd.read_csv(self.data_path + "users.csv")
        self.books = pd.read_csv(self.data_path + "books.csv")
        self.train = pd.read_csv(self.data_path + "train_ratings.csv")
        self.test = pd.read_csv(self.data_path + "test_ratings.csv")
        print("> 데이터 로딩 완료\n")

    def plot_missing_values(self, save_path="eda_missing_values.png"):
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # [Users] 결측치
        users_missing = self.users.isnull().sum()
        users_missing_pct = (users_missing / len(self.users) * 100).sort_values(
            ascending=False
        )

        ax1 = axes[0]
        users_missing_pct.plot(kind="barh", ax=ax1, color="coral")
        ax1.set_title("Users Dataset - Missing Values (%)", fontsize=14, weight="bold")
        ax1.set_xlabel("Missing Percentage (%)")
        for i, v in enumerate(users_missing_pct):
            ax1.text(v + 0.5, i, f"{v:.1f}%", va="center")

        # [Books] 결측치
        books_missing = self.books.isnull().sum()
        books_missing_pct = (books_missing / len(self.books) * 100).sort_values(
            ascending=False
        )

        ax2 = axes[1]
        books_missing_pct.plot(kind="barh", ax=ax2, color="skyblue")
        ax2.set_title("Books Dataset - Missing Values (%)", fontsize=14, weight="bold")
        ax2.set_xlabel("Missing Percentage (%)")
        for i, v in enumerate(books_missing_pct):
            ax2.text(v + 0.5, i, f"{v:.1f}%", va="center")

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 결측치 시각화 저장: {save_path}")
        plt.show()

    # 나이 분포 이상치 탐지
    def plot_age_distribution(self, save_path="eda_age_distribution.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        age_data = self.users["age"].dropna()

        # 1. 전체 분포
        ax1 = axes[0, 0]
        ax1.hist(age_data, bins=50, color="lightblue", edgecolor="black", alpha=0.7)
        ax1.set_title("Age Distribution (All Data)", fontsize=14, weight="bold")
        ax1.set_xlabel("Age")
        ax1.set_ylabel("Frequency")
        ax1.axvline(
            age_data.mean(),
            color="red",
            linestyle="--",
            label=f"Mean: {age_data.mean():.1f}",
        )
        ax1.axvline(
            age_data.median(),
            color="green",
            linestyle="--",
            label=f"Median: {age_data.median():.1f}",
        )
        ax1.legend()

        # 2. 박스 플롯 (이상치 확인)
        ax2 = axes[0, 1]
        ax2.boxplot(
            age_data,
            vert=True,
            patch_artist=True,
            boxprops=dict(facecolor="lightgreen", alpha=0.7),
        )
        ax2.set_title("Age Boxplot - Outlier Detection", fontsize=14, weight="bold")
        ax2.set_ylabel("Age")

        # IQR 기반 이상치 경계선 표시
        Q1, Q3 = age_data.quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        ax2.axhline(
            lower_bound,
            color="red",
            linestyle="--",
            alpha=0.5,
            label=f"Lower: {lower_bound:.1f}",
        )
        ax2.axhline(
            upper_bound,
            color="red",
            linestyle="--",
            alpha=0.5,
            label=f"Upper: {upper_bound:.1f}",
        )
        ax2.legend()

        # 3. 합리적 범위 분포 (0-120세)
        ax3 = axes[1, 0]
        reasonable_age = age_data[(age_data >= 0) & (age_data <= 120)]
        ax3.hist(reasonable_age, bins=50, color="gold", edgecolor="black", alpha=0.7)
        ax3.set_title(
            f"Reasonable Age (0-120) - {len(reasonable_age)} records",
            fontsize=14,
            weight="bold",
        )
        ax3.set_xlabel("Age")
        ax3.set_ylabel("Frequency")

        # 4. 이상치 분석 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        outliers = age_data[(age_data < lower_bound) | (age_data > upper_bound)]
        stats_data = [
            ["Total Records", f"{len(self.users):,}"],
            [
                "Missing Age",
                f'{self.users["age"].isnull().sum():,} ({self.users["age"].isnull().sum()/len(self.users)*100:.1f}%)',
            ],
            ["Valid Age", f"{len(age_data):,}"],
            ["Mean Age", f"{age_data.mean():.2f}"],
            ["Median Age", f"{age_data.median():.2f}"],
            ["Min Age", f"{age_data.min():.2f}"],
            ["Max Age", f"{age_data.max():.2f}"],
            [
                "IQR Outliers",
                f"{len(outliers):,} ({len(outliers)/len(age_data)*100:.1f}%)",
            ],
            ["Age < 0", f"{(age_data < 0).sum()}"],
            ["Age > 120", f"{(age_data > 120).sum()}"],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("Age Statistics Summary", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 나이 분포 저장: {save_path}")
        plt.show()

    # 출판연도 분포 및 이상치 탐지
    def plot_year_distribution(self, save_path="eda_year_distribution.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        year_data = self.books["year_of_publication"].dropna()

        # 1. 전체 분포
        ax1 = axes[0, 0]
        ax1.hist(year_data, bins=100, color="lightcoral", edgecolor="black", alpha=0.7)
        ax1.set_title(
            "Publication Year Distribution (All Data)", fontsize=14, weight="bold"
        )
        ax1.set_xlabel("Year")
        ax1.set_ylabel("Frequency")
        ax1.axvline(
            year_data.median(),
            color="green",
            linestyle="--",
            label=f"Median: {year_data.median():.0f}",
        )
        ax1.legend()

        # 2. 박스플롯
        ax2 = axes[0, 1]
        ax2.boxplot(
            year_data,
            vert=True,
            patch_artist=True,
            boxprops=dict(facecolor="lightblue", alpha=0.7),
        )
        ax2.set_title("Year Boxplot - Outlier Detection", fontsize=14, weight="bold")
        ax2.set_ylabel("Year")

        # 3. 합리적 범위 (1900-2025) 로 임의 설정
        ax3 = axes[1, 0]
        reasonable_year = year_data[(year_data >= 1900) & (year_data <= 2025)]
        ax3.hist(
            reasonable_year,
            bins=50,
            color="mediumseagreen",
            edgecolor="black",
            alpha=0.7,
        )
        ax3.set_title(
            f"Reasonable Year (1900-2025) - {len(reasonable_year)} records",
            fontsize=14,
            weight="bold",
        )
        ax3.set_xlabel("Year")
        ax3.set_ylabel("Frequency")

        # 4. 통계 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Books", f"{len(self.books):,}"],
            ["Valid Years", f"{len(year_data):,}"],
            ["Mean Year", f"{year_data.mean():.0f}"],
            ["Median Year", f"{year_data.median():.0f}"],
            ["Min Year", f"{year_data.min():.0f}"],
            ["Max Year", f"{year_data.max():.0f}"],
            ["Year < 1900", f"{(year_data < 1900).sum():,}"],
            ["Year > 2025", f"{(year_data > 2025).sum():,}"],
            [
                "1900-2025 Range",
                f"{len(reasonable_year):,} ({len(reasonable_year)/len(year_data)*100:.1f}%)",
            ],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("Publication Year Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 출판연도 분포 저장: {save_path}")
        plt.show()

    # 평점 분포
    def plot_rating_distribution(self, save_path="eda_rating_distribution.png"):
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))

        # 1. Train Rating 분포
        ax1 = axes[0]
        rating_counts = self.train["rating"].value_counts().sort_index()
        rating_counts.plot(kind="bar", ax=ax1, color="steelblue", alpha=0.8)
        ax1.set_title("Train Rating Distribution", fontsize=14, weight="bold")
        ax1.set_xlabel("Rating")
        ax1.set_ylabel("Frequency")
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
        for i, v in enumerate(rating_counts):
            ax1.text(i, v + 500, str(v), ha="center", fontsize=9)

        # 2. Rating 박스플롯
        ax2 = axes[1]
        ax2.boxplot(
            self.train["rating"],
            vert=True,
            patch_artist=True,
            boxprops=dict(facecolor="lightgreen", alpha=0.7),
        )
        ax2.set_title("Rating Boxplot", fontsize=14, weight="bold")
        ax2.set_ylabel("Rating")
        ax2.set_ylim(0, 11)

        # 3. 통계 테이블
        ax3 = axes[2]
        ax3.axis("off")

        stats_data = [
            ["Total Ratings", f"{len(self.train):,}"],
            ["Mean Rating", f'{self.train["rating"].mean():.2f}'],
            ["Median Rating", f'{self.train["rating"].median():.1f}'],
            ["Mode Rating", f'{self.train["rating"].mode()[0]}'],
            ["Min Rating", f'{self.train["rating"].min()}'],
            ["Max Rating", f'{self.train["rating"].max()}'],
            ["Std Dev", f'{self.train["rating"].std():.2f}'],
            [
                "Outliers (<1 or >10)",
                f'{((self.train["rating"] < 1) | (self.train["rating"] > 10)).sum()}',
            ],
        ]

        table = ax3.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax3.set_title("Rating Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 평점 분포 저장: {save_path}")
        plt.show()

    # 위치 분석
    def plot_location_analysis(self, save_path="eda_location_analysis.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        def split_location(x):
            res = x.split(",")
            res = [i.strip().lower() for i in res]
            res = [regex.sub(r"[^a-zA-Z/ ]", "", i) for i in res]
            res = [i if i not in ["n/a", ""] else np.nan for i in res]
            res.reverse()
            return res

        # Location 파싱
        self.users["loc_parsed"] = self.users["location"].apply(split_location)
        self.users["loc_country"] = self.users["loc_parsed"].apply(
            lambda x: x[0] if len(x) > 0 else np.nan
        )
        self.users["loc_state"] = self.users["loc_parsed"].apply(
            lambda x: x[1] if len(x) > 1 else np.nan
        )
        self.users["loc_city"] = self.users["loc_parsed"].apply(
            lambda x: x[2] if len(x) > 2 else np.nan
        )

        # 1. Location 결측치
        ax1 = axes[0, 0]
        loc_missing = pd.Series(
            {
                "Country": self.users["loc_country"].isnull().sum(),
                "State": self.users["loc_state"].isnull().sum(),
                "City": self.users["loc_city"].isnull().sum(),
            }
        )
        loc_missing_pct = loc_missing / len(self.users) * 100
        loc_missing_pct.plot(kind="bar", ax=ax1, color=["coral", "gold", "skyblue"])
        ax1.set_title("Location Missing Values (%)", fontsize=14, weight="bold")
        ax1.set_ylabel("Missing Percentage (%)")
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=0)
        for i, v in enumerate(loc_missing_pct):
            ax1.text(i, v + 1, f"{v:.1f}%", ha="center")

        # 2. Top 10 Countries
        ax2 = axes[0, 1]
        top_countries = self.users["loc_country"].value_counts().head(10)
        top_countries.plot(kind="barh", ax=ax2, color="mediumseagreen")
        ax2.set_title("Top 10 Countries", fontsize=14, weight="bold")
        ax2.set_xlabel("User Count")
        for i, v in enumerate(top_countries):
            ax2.text(v + 100, i, str(v), va="center")

        # 3. n/a 패턴 분석
        ax3 = axes[1, 0]
        na_patterns = self.users["location"].value_counts().head(10)
        na_patterns.plot(kind="barh", ax=ax3, color="lightcoral")
        ax3.set_title("Top 10 Location Patterns", fontsize=14, weight="bold")
        ax3.set_xlabel("Count")

        # 4. 통계 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Users", f"{len(self.users):,}"],
            ["Unique Countries", f'{self.users["loc_country"].nunique()}'],
            ["Unique States", f'{self.users["loc_state"].nunique()}'],
            ["Unique Cities", f'{self.users["loc_city"].nunique()}'],
            [
                "Missing Country",
                f'{loc_missing["Country"]:,} ({loc_missing_pct["Country"]:.1f}%)',
            ],
            [
                "Missing State",
                f'{loc_missing["State"]:,} ({loc_missing_pct["State"]:.1f}%)',
            ],
            [
                "Missing City",
                f'{loc_missing["City"]:,} ({loc_missing_pct["City"]:.1f}%)',
            ],
            [
                '"n/a, n/a, n/a"',
                f'{(self.users["location"] == "n/a, n/a, n/a").sum():,}',
            ],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("Location Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 Location 분석 저장: {save_path}")
        plt.show()

    # User-Item 인터랙션 분석
    # for [Cold Start 문제 파악]
    def plot_user_item_interaction(self, save_path="eda_user_item_stats.png"):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))

        # User별 평점 개수
        user_rating_counts = self.train.groupby("user_id").size()

        # Item별 평점 개수
        item_rating_counts = self.train.groupby("isbn").size()

        # 1. User Rating Count 분포
        ax1 = axes[0, 0]
        ax1.hist(
            user_rating_counts, bins=50, color="steelblue", edgecolor="black", alpha=0.7
        )
        ax1.set_title("User Rating Count Distribution", fontsize=14, weight="bold")
        ax1.set_xlabel("Number of Ratings per User")
        ax1.set_ylabel("Frequency")
        ax1.axvline(
            user_rating_counts.median(),
            color="red",
            linestyle="--",
            label=f"Median: {user_rating_counts.median():.0f}",
        )
        ax1.legend()

        # 2. Item Rating Count 분포
        ax2 = axes[0, 1]
        ax2.hist(
            item_rating_counts, bins=50, color="coral", edgecolor="black", alpha=0.7
        )
        ax2.set_title("Book Rating Count Distribution", fontsize=14, weight="bold")
        ax2.set_xlabel("Number of Ratings per Book")
        ax2.set_ylabel("Frequency")
        ax2.axvline(
            item_rating_counts.median(),
            color="red",
            linestyle="--",
            label=f"Median: {item_rating_counts.median():.0f}",
        )
        ax2.legend()

        # 3. Cold Start 문제 (평점 1개 이하)
        ax3 = axes[1, 0]
        cold_start_data = pd.DataFrame(
            {
                "Users (≤5 ratings)": [
                    (user_rating_counts <= i).sum() for i in [1, 3, 5]
                ],
                "Books (≤5 ratings)": [
                    (item_rating_counts <= i).sum() for i in [1, 3, 5]
                ],
            },
            index=["≤1", "≤3", "≤5"],
        )

        cold_start_data.plot(kind="bar", ax=ax3, color=["steelblue", "coral"])
        ax3.set_title("Cold Start Problem Analysis", fontsize=14, weight="bold")
        ax3.set_ylabel("Count")
        ax3.set_xticklabels(ax3.get_xticklabels(), rotation=0)
        ax3.legend()

        # 4. 통계 테이블
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Users", f'{self.train["user_id"].nunique():,}'],
            ["Total Books", f'{self.train["isbn"].nunique():,}'],
            ["Total Ratings", f"{len(self.train):,}"],
            ["Avg Ratings/User", f"{user_rating_counts.mean():.2f}"],
            ["Avg Ratings/Book", f"{item_rating_counts.mean():.2f}"],
            ["Users w/ 1 rating", f"{(user_rating_counts == 1).sum():,}"],
            ["Books w/ 1 rating", f"{(item_rating_counts == 1).sum():,}"],
            [
                "Sparsity",
                f'{(1 - len(self.train) / (self.train["user_id"].nunique() * self.train["isbn"].nunique())) * 100:.2f}%',
            ],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 2)
        ax4.set_title("User-Item Statistics", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 User-Item 인터랙션 저장: {save_path}")
        plt.show()

    def plot_category_distribution(self, save_path="eda_category_distribution.png"):
        """Book Category 분포 시각화"""
        fig, axes = plt.subplots(2, 2, figsize=(20, 16))

        # Category 결측치 확인
        missing = self.books["category"].isna().sum()
        books_with_cat = self.books[self.books["category"].notna()].copy()

        print(f"\n> Category 분석:")
        print(f"  - 총 책: {len(self.books):,}권")
        print(f"  - 결측치: {missing:,}개 ({missing/len(self.books)*100:.2f}%)")
        print(f"  - 유효 데이터: {len(books_with_cat):,}권")

        # Category 분포 계산
        cat_counts = books_with_cat["category"].value_counts()
        print(f"  - 고유 Category: {len(cat_counts):,}개")

        # 1. Top 30 Category Bar Chart
        ax1 = axes[0, 0]
        top_30 = cat_counts.head(30)
        bars = ax1.barh(
            range(len(top_30)), top_30.values, color="steelblue", edgecolor="black"
        )
        ax1.set_yticks(range(len(top_30)))
        ax1.set_yticklabels(top_30.index, fontsize=9)
        ax1.invert_yaxis()
        ax1.set_xlabel("Number of Books", fontsize=12, fontweight="bold")
        ax1.set_title("Top 30 Book Categories", fontsize=14, fontweight="bold", pad=20)
        ax1.grid(axis="x", alpha=0.3)

        # 값 표시
        for i, (idx, val) in enumerate(top_30.items()):
            ax1.text(val + 50, i, f"{val:,}", va="center", fontsize=8)

        # 2. Category Size Distribution (Log Scale)
        ax2 = axes[0, 1]
        bins = np.logspace(0, np.log10(cat_counts.max()), 50)
        ax2.hist(
            cat_counts.values,
            bins=bins,
            color="coral",
            edgecolor="black",
            alpha=0.7,
        )
        ax2.set_xscale("log")
        ax2.set_xlabel(
            "Number of Books per Category (log scale)",
            fontsize=12,
            fontweight="bold",
        )
        ax2.set_ylabel("Number of Categories", fontsize=12, fontweight="bold")
        ax2.set_title(
            "Category Size Distribution (Log Scale)",
            fontsize=14,
            fontweight="bold",
            pad=20,
        )
        ax2.grid(True, alpha=0.3)

        # 통계선 추가
        ax2.axvline(
            cat_counts.mean(),
            color="red",
            linestyle="--",
            linewidth=2,
            label=f"Mean: {cat_counts.mean():.0f}",
        )
        ax2.axvline(
            cat_counts.median(),
            color="green",
            linestyle="--",
            linewidth=2,
            label=f"Median: {cat_counts.median():.0f}",
        )
        ax2.legend(fontsize=10)

        # 3. Cumulative Distribution (Pareto Chart)
        ax3 = axes[1, 0]
        cumsum = cat_counts.sort_values(ascending=False).cumsum()
        cumsum_pct = cumsum / cumsum.iloc[-1] * 100
        ax3.plot(
            range(len(cumsum_pct)),
            cumsum_pct.values,
            linewidth=2,
            color="darkgreen",
        )
        ax3.fill_between(
            range(len(cumsum_pct)), cumsum_pct.values, alpha=0.3, color="lightgreen"
        )
        ax3.set_xlabel(
            "Number of Categories (sorted by size)", fontsize=12, fontweight="bold"
        )
        ax3.set_ylabel("Cumulative % of Books", fontsize=12, fontweight="bold")
        ax3.set_title(
            "Cumulative Distribution (Pareto Chart)",
            fontsize=14,
            fontweight="bold",
            pad=20,
        )
        ax3.grid(True, alpha=0.3)

        # 80% 선 표시
        idx_80 = (cumsum_pct >= 80).idxmax()
        ax3.axhline(80, color="red", linestyle="--", linewidth=2, alpha=0.7)
        ax3.axvline(idx_80, color="red", linestyle="--", linewidth=2, alpha=0.7)
        ax3.text(
            idx_80,
            85,
            f"Top {int(idx_80)} categories\n= 80% of books",
            fontsize=10,
            bbox=dict(boxstyle="round", facecolor="yellow", alpha=0.7),
        )

        # 4. Statistics Table
        ax4 = axes[1, 1]
        ax4.axis("off")

        stats_data = [
            ["Total Books", f"{len(self.books):,}"],
            [
                "Missing Category",
                f"{missing:,} ({missing/len(self.books)*100:.1f}%)",
            ],
            ["Books with Category", f"{len(books_with_cat):,}"],
            ["Unique Categories", f"{len(cat_counts):,}"],
            ["", ""],
            ["Mean books/category", f"{cat_counts.mean():.1f}"],
            ["Median books/category", f"{cat_counts.median():.0f}"],
            ["Max books/category", f"{cat_counts.max():,}"],
            ["Min books/category", f"{cat_counts.min()}"],
            ["", ""],
            ["1 book only", f"{(cat_counts == 1).sum():,}"],
            ["2-9 books", f"{((cat_counts >= 2) & (cat_counts < 10)).sum():,}"],
            ["10-99 books", f"{((cat_counts >= 10) & (cat_counts < 100)).sum():,}"],
            [
                "100-999 books",
                f"{((cat_counts >= 100) & (cat_counts < 1000)).sum():,}",
            ],
            ["1000+ books", f"{(cat_counts >= 1000).sum():,}"],
        ]

        table = ax4.table(
            cellText=stats_data,
            colLabels=["Metric", "Value"],
            cellLoc="left",
            loc="center",
            colWidths=[0.6, 0.4],
        )
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1, 2.5)

        # 헤더 스타일
        for i in range(2):
            table[(0, i)].set_facecolor("#4472C4")
            table[(0, i)].set_text_props(weight="bold", color="white")

        ax4.set_title("Category Statistics Summary", fontsize=14, weight="bold", pad=20)

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 카테고리 분포 저장: {save_path}")
        plt.show()

    def plot_category_longtail(self, save_path="eda_category_longtail.png"):
        """Category Long Tail 분석"""
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))

        books_with_cat = self.books[self.books["category"].notna()].copy()
        cat_counts = books_with_cat["category"].value_counts()
        sorted_cats = cat_counts.sort_values(ascending=False)

        # 1. Long Tail Visualization
        ax1 = axes[0]
        x_range = range(len(sorted_cats))
        ax1.bar(
            x_range,
            sorted_cats.values,
            color="steelblue",
            alpha=0.7,
            edgecolor="none",
        )
        ax1.set_xlabel("Category Rank", fontsize=12, fontweight="bold")
        ax1.set_ylabel("Number of Books", fontsize=12, fontweight="bold")
        ax1.set_title(
            "Category Long Tail Distribution",
            fontsize=14,
            fontweight="bold",
            pad=20,
        )
        ax1.grid(True, alpha=0.3)

        # Head, Middle, Tail 구분선
        head_idx = int(len(sorted_cats) * 0.2)  # Top 20%
        tail_idx = int(len(sorted_cats) * 0.8)  # Bottom 20%

        ax1.axvline(
            head_idx,
            color="red",
            linestyle="--",
            linewidth=2,
            alpha=0.7,
            label=f"Head (Top 20%): {int(head_idx)} categories",
        )
        ax1.axvline(
            tail_idx,
            color="orange",
            linestyle="--",
            linewidth=2,
            alpha=0.7,
            label=f"Tail (Bottom 20%): {int(len(sorted_cats)-tail_idx)} categories",
        )
        ax1.legend(fontsize=10)

        # 2. Head vs Middle vs Tail Comparison
        ax2 = axes[1]

        head_cats = sorted_cats.iloc[:head_idx]
        middle_cats = sorted_cats.iloc[head_idx:tail_idx]
        tail_cats = sorted_cats.iloc[tail_idx:]

        comparison_data = pd.DataFrame(
            {
                "Categories": [len(head_cats), len(middle_cats), len(tail_cats)],
                "Total Books": [
                    head_cats.sum(),
                    middle_cats.sum(),
                    tail_cats.sum(),
                ],
                "Avg Books/Cat": [
                    head_cats.mean(),
                    middle_cats.mean(),
                    tail_cats.mean(),
                ],
            },
            index=["Head (Top 20%)", "Middle (20-80%)", "Tail (Bottom 20%)"],
        )

        x = np.arange(len(comparison_data.index))
        width = 0.25

        bars1 = ax2.bar(
            x - width,
            comparison_data["Categories"],
            width,
            label="Categories",
            color="steelblue",
            alpha=0.8,
        )
        bars2 = ax2.bar(
            x,
            comparison_data["Total Books"] / 100,
            width,
            label="Total Books (÷100)",
            color="coral",
            alpha=0.8,
        )
        bars3 = ax2.bar(
            x + width,
            comparison_data["Avg Books/Cat"],
            width,
            label="Avg Books/Cat",
            color="lightgreen",
            alpha=0.8,
        )

        ax2.set_xlabel("Category Group", fontsize=12, fontweight="bold")
        ax2.set_ylabel("Count", fontsize=12, fontweight="bold")
        ax2.set_title(
            "Head vs Middle vs Tail Comparison",
            fontsize=14,
            fontweight="bold",
            pad=20,
        )
        ax2.set_xticks(x)
        ax2.set_xticklabels(comparison_data.index)
        ax2.legend()
        ax2.grid(axis="y", alpha=0.3)

        # 값 표시
        for bars in [bars1, bars2, bars3]:
            for bar in bars:
                height = bar.get_height()
                ax2.text(
                    bar.get_x() + bar.get_width() / 2.0,
                    height,
                    f"{int(height):,}",
                    ha="center",
                    va="bottom",
                    fontsize=8,
                )

        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"💾 카테고리 Long Tail 분석 저장: {save_path}")
        plt.show()

    # 전체 EDA 리포트 생성
    def generate_full_report(self):
        print("=" * 60)
        print("EDA 시각화 시작")
        print("=" * 60)

        self.load_data()

        print("\n1) 결측치 분석 중...")
        self.plot_missing_values()

        print("\n2) 나이 분포 분석 중...")
        self.plot_age_distribution()

        print("\n3) 출판연도 분포 분석 중...")
        self.plot_year_distribution()

        print("\n4) 평점 분포 분석 중...")
        self.plot_rating_distribution()

        print("\n5) 위치 분석 중...")
        self.plot_location_analysis()

        print("\n6) User-Item 인터랙션 분석 중...")
        self.plot_user_item_interaction()

        print("\n" + "=" * 60)
        print("EDA 시각화 완료")
        print("=" * 60)

## 사용

In [11]:
if __name__ == "__main__":
    DATA_PATH = "/data/ephemeral/home/sojin/data/"

    visualizer = EDAVisualizer(DATA_PATH)
    visualizer.generate_full_report()

EDA 시각화 시작
> 데이터 로딩 중...
> 데이터 로딩 완료


1) 결측치 분석 중...
💾 결측치 시각화 저장: eda_missing_values.png

2) 나이 분포 분석 중...
💾 나이 분포 저장: eda_age_distribution.png

3) 출판연도 분포 분석 중...
💾 출판연도 분포 저장: eda_year_distribution.png

4) 평점 분포 분석 중...
💾 평점 분포 저장: eda_rating_distribution.png

5) 위치 분석 중...
💾 Location 분석 저장: eda_location_analysis.png

6) User-Item 인터랙션 분석 중...
💾 User-Item 인터랙션 저장: eda_user_item_stats.png

EDA 시각화 완료


In [13]:
"""
📚 Book Category 분포 시각화

원본 데이터의 카테고리 분포를 분석하고 시각화합니다.

사용법: 주피터 노트북에서 셀 단위로 실행
"""

# ============================================================================
# 1. 라이브러리 임포트
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정 (선택사항)
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False

# 스타일 설정
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (15, 10)

print("✅ 라이브러리 로드 완료")


# ============================================================================
# 2. 데이터 로드
# ============================================================================

# 데이터 경로 설정
DATA_PATH = "/data/ephemeral/home/sojin/data/"

# Books 데이터 로드
books = pd.read_csv(DATA_PATH + "books.csv")

print("=" * 80)
print("📚 데이터 로드 완료")
print("=" * 80)
print(f"총 책: {len(books):,}권")
print(f"컬럼: {list(books.columns)}")


# ============================================================================
# 3. Category 기본 통계
# ============================================================================

# Category 결측치 확인
missing = books["category"].isna().sum()
books_with_cat = books[books["category"].notna()].copy()

print("\n📊 Category 기본 통계")
print("=" * 80)
print(f"결측치: {missing:,}개 ({missing/len(books)*100:.2f}%)")
print(f"유효 데이터: {len(books_with_cat):,}권")

# Category 분포 계산
cat_counts = books_with_cat["category"].value_counts()
print(f"\n고유 Category: {len(cat_counts):,}개")
print(f"최다 Category: {cat_counts.index[0]} ({cat_counts.iloc[0]:,}권)")
print(f"최소 Category: {cat_counts.index[-1]} ({cat_counts.iloc[-1]}권)")

# 통계 요약
print(f"\n평균 책 수/Category: {cat_counts.mean():.1f}권")
print(f"중앙값: {cat_counts.median():.0f}권")
print(f"표준편차: {cat_counts.std():.1f}권")

print(f"\n1권만 있는 Category: {(cat_counts == 1).sum():,}개")
print(f"10권 이상 Category: {(cat_counts >= 10).sum():,}개")
print(f"100권 이상 Category: {(cat_counts >= 100).sum():,}개")
print(f"1000권 이상 Category: {(cat_counts >= 1000).sum():,}개")


# ============================================================================
# 4. Top 30 Category 확인
# ============================================================================

# Top 30 카테고리 DataFrame으로 출력
top_30_df = pd.DataFrame(
    {
        "Category": cat_counts.head(30).index,
        "Count": cat_counts.head(30).values,
        "Percentage": (cat_counts.head(30).values / len(books_with_cat) * 100).round(2),
    }
)

print("\n📊 Top 30 Categories")
print("=" * 80)
display(top_30_df)


# ============================================================================
# 5. 시각화 1: Category Distribution (종합)
# ============================================================================

print("\n📈 시각화 1 생성 중...")

fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# 1-1. Top 30 Category Bar Chart
ax1 = axes[0, 0]
top_30 = cat_counts.head(30)
bars = ax1.barh(range(len(top_30)), top_30.values, color="steelblue", edgecolor="black")
ax1.set_yticks(range(len(top_30)))
ax1.set_yticklabels(top_30.index, fontsize=9)
ax1.invert_yaxis()
ax1.set_xlabel("Number of Books", fontsize=12, fontweight="bold")
ax1.set_title("Top 30 Book Categories", fontsize=14, fontweight="bold", pad=20)
ax1.grid(axis="x", alpha=0.3)

for i, (idx, val) in enumerate(top_30.items()):
    ax1.text(val + 50, i, f"{val:,}", va="center", fontsize=8)

# 1-2. Category Size Distribution (Log Scale)
ax2 = axes[0, 1]
bins = np.logspace(0, np.log10(cat_counts.max()), 50)
ax2.hist(cat_counts.values, bins=bins, color="coral", edgecolor="black", alpha=0.7)
ax2.set_xscale("log")
ax2.set_xlabel(
    "Number of Books per Category (log scale)", fontsize=12, fontweight="bold"
)
ax2.set_ylabel("Number of Categories", fontsize=12, fontweight="bold")
ax2.set_title(
    "Category Size Distribution (Log Scale)", fontsize=14, fontweight="bold", pad=20
)
ax2.grid(True, alpha=0.3)

ax2.axvline(
    cat_counts.mean(),
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Mean: {cat_counts.mean():.0f}",
)
ax2.axvline(
    cat_counts.median(),
    color="green",
    linestyle="--",
    linewidth=2,
    label=f"Median: {cat_counts.median():.0f}",
)
ax2.legend(fontsize=10)

# 1-3. Cumulative Distribution (Pareto Chart)
ax3 = axes[1, 0]
cumsum = cat_counts.sort_values(ascending=False).cumsum()
cumsum_pct = cumsum / cumsum.iloc[-1] * 100
ax3.plot(range(len(cumsum_pct)), cumsum_pct.values, linewidth=2, color="darkgreen")
ax3.fill_between(
    range(len(cumsum_pct)), cumsum_pct.values, alpha=0.3, color="lightgreen"
)
ax3.set_xlabel("Number of Categories (sorted by size)", fontsize=12, fontweight="bold")
ax3.set_ylabel("Cumulative % of Books", fontsize=12, fontweight="bold")
ax3.set_title(
    "Cumulative Distribution (Pareto Chart)", fontsize=14, fontweight="bold", pad=20
)
ax3.grid(True, alpha=0.3)

# 80% 지점의 인덱스 번호 찾기 (위치)
idx_80 = (cumsum_pct >= 80).argmax()  # argmax()로 변경!
ax3.axhline(80, color="red", linestyle="--", linewidth=2, alpha=0.7)
ax3.axvline(idx_80, color="red", linestyle="--", linewidth=2, alpha=0.7)
ax3.text(
    idx_80,
    85,
    f"Top {idx_80} categories\n= 80% of books",
    fontsize=10,
    bbox=dict(boxstyle="round", facecolor="yellow", alpha=0.7),
)

# 1-4. Statistics Table
ax4 = axes[1, 1]
ax4.axis("off")

stats_data = [
    ["Total Books", f"{len(books):,}"],
    ["Missing Category", f"{missing:,} ({missing/len(books)*100:.1f}%)"],
    ["Books with Category", f"{len(books_with_cat):,}"],
    ["Unique Categories", f"{len(cat_counts):,}"],
    ["", ""],
    ["Mean books/category", f"{cat_counts.mean():.1f}"],
    ["Median books/category", f"{cat_counts.median():.0f}"],
    ["Max books/category", f"{cat_counts.max():,}"],
    ["Min books/category", f"{cat_counts.min()}"],
    ["", ""],
    ["1 book only", f"{(cat_counts == 1).sum():,}"],
    ["2-9 books", f"{((cat_counts >= 2) & (cat_counts < 10)).sum():,}"],
    ["10-99 books", f"{((cat_counts >= 10) & (cat_counts < 100)).sum():,}"],
    ["100-999 books", f"{((cat_counts >= 100) & (cat_counts < 1000)).sum():,}"],
    ["1000+ books", f"{(cat_counts >= 1000).sum():,}"],
]

table = ax4.table(
    cellText=stats_data,
    colLabels=["Metric", "Value"],
    cellLoc="left",
    loc="center",
    colWidths=[0.6, 0.4],
)
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

for i in range(2):
    table[(0, i)].set_facecolor("#4472C4")
    table[(0, i)].set_text_props(weight="bold", color="white")

ax4.set_title("Category Statistics Summary", fontsize=14, weight="bold", pad=20)

plt.tight_layout()
plt.savefig("eda_category_distribution.png", dpi=300, bbox_inches="tight")
print("💾 저장: eda_category_distribution.png")
plt.show()


# ============================================================================
# 6. 시각화 2: Long Tail Analysis
# ============================================================================

print("\n📈 시각화 2 생성 중...")

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sorted_cats = cat_counts.sort_values(ascending=False)

# 2-1. Long Tail Visualization
ax1 = axes[0]
x_range = range(len(sorted_cats))
ax1.bar(x_range, sorted_cats.values, color="steelblue", alpha=0.7, edgecolor="none")
ax1.set_xlabel("Category Rank", fontsize=12, fontweight="bold")
ax1.set_ylabel("Number of Books", fontsize=12, fontweight="bold")
ax1.set_title("Category Long Tail Distribution", fontsize=14, fontweight="bold", pad=20)
ax1.grid(True, alpha=0.3)

head_idx = int(len(sorted_cats) * 0.2)
tail_idx = int(len(sorted_cats) * 0.8)

ax1.axvline(
    head_idx,
    color="red",
    linestyle="--",
    linewidth=2,
    alpha=0.7,
    label=f"Head (Top 20%): {int(head_idx)} categories",
)
ax1.axvline(
    tail_idx,
    color="orange",
    linestyle="--",
    linewidth=2,
    alpha=0.7,
    label=f"Tail (Bottom 20%): {int(len(sorted_cats)-tail_idx)} categories",
)
ax1.legend(fontsize=10)

# 2-2. Head vs Middle vs Tail Comparison
ax2 = axes[1]

head_cats = sorted_cats.iloc[:head_idx]
middle_cats = sorted_cats.iloc[head_idx:tail_idx]
tail_cats = sorted_cats.iloc[tail_idx:]

comparison_data = pd.DataFrame(
    {
        "Categories": [len(head_cats), len(middle_cats), len(tail_cats)],
        "Total Books": [head_cats.sum(), middle_cats.sum(), tail_cats.sum()],
        "Avg Books/Cat": [head_cats.mean(), middle_cats.mean(), tail_cats.mean()],
    },
    index=["Head (Top 20%)", "Middle (20-80%)", "Tail (Bottom 20%)"],
)

x = np.arange(len(comparison_data.index))
width = 0.25

bars1 = ax2.bar(
    x - width,
    comparison_data["Categories"],
    width,
    label="Categories",
    color="steelblue",
    alpha=0.8,
)
bars2 = ax2.bar(
    x,
    comparison_data["Total Books"] / 100,
    width,
    label="Total Books (÷100)",
    color="coral",
    alpha=0.8,
)
bars3 = ax2.bar(
    x + width,
    comparison_data["Avg Books/Cat"],
    width,
    label="Avg Books/Cat",
    color="lightgreen",
    alpha=0.8,
)

ax2.set_xlabel("Category Group", fontsize=12, fontweight="bold")
ax2.set_ylabel("Count", fontsize=12, fontweight="bold")
ax2.set_title(
    "Head vs Middle vs Tail Comparison", fontsize=14, fontweight="bold", pad=20
)
ax2.set_xticks(x)
ax2.set_xticklabels(comparison_data.index)
ax2.legend()
ax2.grid(axis="y", alpha=0.3)

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax2.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{int(height):,}",
            ha="center",
            va="bottom",
            fontsize=8,
        )

plt.tight_layout()
plt.savefig("eda_category_longtail.png", dpi=300, bbox_inches="tight")
print("💾 저장: eda_category_longtail.png")
plt.show()


# ============================================================================
# 7. Long Tail 분석 요약
# ============================================================================

# Long Tail 분석 DataFrame
longtail_summary = pd.DataFrame(
    {
        "Group": ["Head (Top 20%)", "Middle (20-80%)", "Tail (Bottom 20%)"],
        "Categories": [len(head_cats), len(middle_cats), len(tail_cats)],
        "Total Books": [head_cats.sum(), middle_cats.sum(), tail_cats.sum()],
        "Avg Books/Cat": [
            round(head_cats.mean(), 1),
            round(middle_cats.mean(), 1),
            round(tail_cats.mean(), 1),
        ],
        "% of Total Books": [
            round(head_cats.sum() / len(books_with_cat) * 100, 2),
            round(middle_cats.sum() / len(books_with_cat) * 100, 2),
            round(tail_cats.sum() / len(books_with_cat) * 100, 2),
        ],
    }
)

print("\n" + "=" * 80)
print("📊 Long Tail 분석 요약")
print("=" * 80)
display(longtail_summary)

print("\n✅ 모든 시각화 완료!")
print("\n생성된 파일:")
print("  1. eda_category_distribution.png")
print("  2. eda_category_longtail.png")

✅ 라이브러리 로드 완료
📚 데이터 로드 완료
총 책: 149,570권
컬럼: ['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher', 'img_url', 'language', 'category', 'summary', 'img_path']

📊 Category 기본 통계
결측치: 68,851개 (46.03%)
유효 데이터: 80,719권

고유 Category: 4,292개
최다 Category: ['Fiction'] (32,956권)
최소 Category: ['Vocational guidance'] (1권)

평균 책 수/Category: 18.8권
중앙값: 1권
표준편차: 518.3권

1권만 있는 Category: 2,710개
10권 이상 Category: 254개
100권 이상 Category: 51개
1000권 이상 Category: 11개

📊 Top 30 Categories


,Category,Count,Percentage
0,['Fiction'],32956,40.83
1,['Juvenile Fiction'],5804,7.19
2,['Biography & Autobiography'],3320,4.11
3,['History'],1925,2.38
4,['Religion'],1818,2.25
5,['Juvenile Nonfiction'],1417,1.76
6,['Social Science'],1231,1.53
7,['Humor'],1161,1.44
8,"['Body, Mind & Spirit']",1109,1.37
9,['Business & Economics'],1070,1.33



📈 시각화 1 생성 중...
💾 저장: eda_category_distribution.png

📈 시각화 2 생성 중...
💾 저장: eda_category_longtail.png

📊 Long Tail 분석 요약


,Group,Categories,Total Books,Avg Books/Cat,% of Total Books
0,Head (Top 20%),858,76439,89.1,94.70
1,Middle (20-80%),2575,3421,1.3,4.24
2,Tail (Bottom 20%),859,859,1.0,1.06



✅ 모든 시각화 완료!

생성된 파일:
  1. eda_category_distribution.png
  2. eda_category_longtail.png
